In [1]:
import nltk
import re
#nltk.download('stopwords')
import pickle
#from nltk.corpus import stopwords

In [2]:
def explode(df, lst_cols, fill_value='', preserve_index=False):
    # make sure `lst_cols` is list-alike
    if (lst_cols is not None
        and len(lst_cols) > 0
        and not isinstance(lst_cols, (list, tuple, np.ndarray, pd.Series))):
        lst_cols = [lst_cols]
    # all columns except `lst_cols`
    idx_cols = df.columns.difference(lst_cols)
    # calculate lengths of lists
    lens = df[lst_cols[0]].str.len()
    # preserve original index values    
    idx = np.repeat(df.index.values, lens)
    # create "exploded" DF
    res = (pd.DataFrame({
                col:np.repeat(df[col].values, lens)
                for col in idx_cols},
                index=idx)
             .assign(**{col:np.concatenate(df.loc[lens>0, col].values)
                            for col in lst_cols}))
    # append those rows that have empty lists
    if (lens == 0).any():
        # at least one list in cells is empty
        res = (res.append(df.loc[lens==0, idx_cols], sort=False)
                  .fillna(fill_value))
    # revert the original index order
    res = res.sort_index()
    # reset index if requested
    if not preserve_index:        
        res = res.reset_index(drop=True)
    return res

In [3]:
from sklearn.datasets import load_files
my_data=load_files(r'\\lsv16171.live.mdu.local\home$\zz2841\Documents\ML Docs\document classification')
X,y=my_data.data,my_data.target

In [4]:
import pandas as pd
import numpy as np
df=pd.DataFrame({'filenames':[my_data.filenames],'target':[my_data.target]})
df=explode(df,['filenames','target'],fill_value='',preserve_index=True)
df['target']=df['target'].replace([0,1,2,3,4],['3rd Party Expert','GMC','MDU Invoices','NHSLA','Solicitor Contacts'])
df.to_csv(r'\\lsv16171.live.mdu.local\home$\zz2841\Documents\ML Docs\document classification\my_data.csv',index=False)
df

,filenames,target
0,\\lsv16171.live.mdu.local\home$\zz2841\Documen...,3rd Party Expert
0,\\lsv16171.live.mdu.local\home$\zz2841\Documen...,MDU Invoices
0,\\lsv16171.live.mdu.local\home$\zz2841\Documen...,Solicitor Contacts
0,\\lsv16171.live.mdu.local\home$\zz2841\Documen...,MDU Invoices
0,\\lsv16171.live.mdu.local\home$\zz2841\Documen...,3rd Party Expert
...,...,...
0,\\lsv16171.live.mdu.local\home$\zz2841\Documen...,GMC
0,\\lsv16171.live.mdu.local\home$\zz2841\Documen...,GMC
0,\\lsv16171.live.mdu.local\home$\zz2841\Documen...,MDU Invoices
0,\\lsv16171.live.mdu.local\home$\zz2841\Documen...,Solicitor Contacts


In [5]:
'''txt cleaning on test data'''
import re
documents=[]
from nltk.stem import WordNetLemmatizer
stemmer=WordNetLemmatizer()
for sen in range(0,len(X)):
    document=re.sub(r'\W','',str(X[sen]))
    document=re.sub(r'\s+[a-zA-Z]\s+','',document)
    document=re.sub(r'\s^[a-zA-Z]\s+','',document)
    document=re.sub(r'\s+','',document,flags=re.I)
    document=re.sub(r'^b\s+','',document)
    document=document.lower()
    document=document.split()
    # document=[stemmer.lemmatize(word) for word in document]
    document=''.join(document)
    documents.append(document)  

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(X)

In [7]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidfconvertor=TfidfTransformer()
X=tfidfconvertor.fit_transform(X).toarray()

In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,shuffle=True)

In [9]:
'''training the data in RF model'''
from sklearn.ensemble._forest import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
classifier=RandomForestClassifier()
classifier.fit(X_train,y_train)
y_pred=classifier.predict(X_test)
print(accuracy_score(y_test,y_pred))

0.8224455611390284


In [10]:
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test,y_pred))

[[110   1   2   9   3]
 [ 19 115   0   9   1]
 [  1   1 116   0   4]
 [ 10   8   0  98   2]
 [ 10   8   3  15  52]]
              precision    recall  f1-score   support

           0       0.73      0.88      0.80       125
           1       0.86      0.80      0.83       144
           2       0.96      0.95      0.95       122
           3       0.75      0.83      0.79       118
           4       0.84      0.59      0.69        88

    accuracy                           0.82       597
   macro avg       0.83      0.81      0.81       597
weighted avg       0.83      0.82      0.82       597

0.8224455611390284
